In [ ]:
#Installing FinRL
%%capture
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [ ]:
%%capture
!pip install "ray[tune]" 

In [ ]:
%%capture
!pip install GPy

#Importing libraries

In [ ]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
%matplotlib inline
# from finrl import config
import finrl
# from finrl import finrl_meta
# from finrl import applications
# from finrl import agents
from finrl import config, config_tickers
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading_np import StockTradingEnv as StockTradingEnv_numpy 
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.rllib.models import DRLAgent as DRLAgent_rllib
from finrl.meta.data_processor import DataProcessor
# from stable_baselines3.common.vec_env import DummyVecEnv
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
import ray
from pprint import pprint
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.ddpg import DDPGTrainer
from ray.rllib.agents.a3c import A2CTrainer
from ray.rllib.agents.a3c import a2c
from ray.rllib.agents.ddpg import ddpg, td3
from ray.rllib.agents.ppo import ppo
from ray.rllib.agents.sac import sac
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.schedulers.pb2 import PB2
from ray.tune import run, sample_from
from ray.tune.registry import register_env

import sys
import os
import itertools
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter

from ray.tune.registry import register_env

import time
from typing import Dict, Optional, Any
import psutil
psutil_memory_in_bytes = psutil.virtual_memory().total

ray._private.utils.get_system_memory = lambda: psutil_memory_in_bytes


/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [ ]:
MODELS = {"a2c": a2c, "ddpg": ddpg, "td3": td3, "sac": sac, "ppo": ppo}

## Helper functions for train and test environments

In [ ]:
#Function to calculate the sharpe ratio from the list of total_episode_reward
def calculate_sharpe(episode_reward:list):
  perf_data = pd.DataFrame(data=episode_reward,columns=['reward'])
  perf_data['daily_return'] = perf_data['reward'].pct_change(1)
  if perf_data['daily_return'].std() !=0:
    sharpe = (252**0.5)*perf_data['daily_return'].mean()/ \
          perf_data['daily_return'].std()
    return sharpe
  else:
    return 0

In [ ]:
TRAIN_START_DATE = '2014-01-01'
TRAIN_END_DATE = '2019-07-30'

VAL_START_DATE = '2019-08-01'
VAL_END_DATE = '2020-07-30'

TEST_START_DATE = '2020-08-01'
TEST_END_DATE = '2021-10-01'

In [ ]:
from finrl.config_tickers import DOW_30_TICKER
technical_indicator_list =config.INDICATORS

model_name = 'ppo'
env = StockTradingEnv_numpy
# ticker_list = ['SPY','TSLA','AAPL','GOOGL']
ticker_list = DOW_30_TICKER
data_source = 'yahoofinance'
time_interval = '1D'

In [ ]:
def get_train_env(start_date, end_date, ticker_list, data_source, time_interval, 
          technical_indicator_list, env, model_name, if_vix = True,
          **kwargs):
    
    #fetch data
    DP = DataProcessor(data_source, **kwargs)
    data = DP.download_data(ticker_list, start_date, end_date, time_interval)
    data = DP.clean_data(data)
    data = DP.add_technical_indicator(data, technical_indicator_list)
    if if_vix:
        data = DP.add_vix(data)
    price_array, tech_array, turbulence_array = DP.df_to_array(data, if_vix)
    train_env_config = {'price_array':price_array,
              'tech_array':tech_array,
              'turbulence_array':turbulence_array,
              'if_train':True}
    
    return train_env_config

def calculate_sharpe(episode_reward:list):
  perf_data = pd.DataFrame(data=episode_reward,columns=['reward'])
  perf_data['daily_return'] = perf_data['reward'].pct_change(1)
  if perf_data['daily_return'].std() !=0:
    sharpe = (252**0.5)*perf_data['daily_return'].mean()/ \
          perf_data['daily_return'].std()
    return sharpe
  else:
    return 0

def get_test_config(start_date, end_date, ticker_list, data_source, time_interval, 
         technical_indicator_list, env, model_name, if_vix = True,
         **kwargs):
  
  DP = DataProcessor(data_source, **kwargs)
  data = DP.download_data(ticker_list, start_date, end_date, time_interval)
  data = DP.clean_data(data)
  data = DP.add_technical_indicator(data, technical_indicator_list)
  
  if if_vix:
      data = DP.add_vix(data)
  
  price_array, tech_array, turbulence_array = DP.df_to_array(data, if_vix)
  test_env_config = {'price_array':price_array,
            'tech_array':tech_array,
            'turbulence_array':turbulence_array,'if_train':False}
  return test_env_config

def val_or_test(test_env_config,agent_path,model_name,env):
  episode_total_reward = DRL_prediction(model_name,test_env_config,
                                env = env,
                                agent_path=agent_path)


  return calculate_sharpe(episode_total_reward),episode_total_reward

In [ ]:
train_env_config = get_train_env(TRAIN_START_DATE, VAL_END_DATE, 
                     ticker_list, data_source, time_interval, 
                        technical_indicator_list, env, model_name)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

## Registering for an environment

In [ ]:
from ray.tune.registry import register_env
env = StockTradingEnv_numpy
env_name = 'StockTrading_train_env'
register_env(env_name, lambda config: env(train_env_config))

##Defining hyperparameter search space

1. You can look up [here](https://docs.ray.io/en/latest/tune/key-concepts.html#search-spaces) to learn how to define hyperparameter search space
2. Jump over to this [link](https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/utils/hyperparams_opt.py) to find the range of different hyperparameter
3. To learn about different hyperparameters for different algorithms for RLlib models, jump over to this [link](https://docs.ray.io/en/latest/rllib-algorithms.html)

In [ ]:
import numpy as np

def loguniform(low=0,high=1):
  return np.exp(np.random.uniform(low,high))

def sample_ddpg_params():
  
  return {
  "buffer_size": tune.choice([int(1e4), int(1e5), int(1e6)]),
  "lr": tune.loguniform(1e-5, 1),
  "train_batch_size": tune.choice([32, 64, 128, 256, 512])
  }
def sample_a2c_params():
  
  return{
       "lambda": tune.choice([0.1,0.3,0.5,0.7,0.9,1.0]),
      "entropy_coeff": tune.loguniform(0.00000001, 0.1),
      "lr": tune.loguniform(1e-5, 1)    
  }

def sample_ppo_params():
  return {
     'lr':tune.loguniform(5e-5,0.0001),
      'sgd_minibatch_size': tune.choice([64,128,256]),
      'entropy_coeff': tune.loguniform(0.00000001, 0.1),
        'clip_param':tune.choice([0.1,0.2,0.3,0.4]),
        "vf_loss_coeff": tune.uniform(0,1),
        'lambda': tune.choice([0.9, 0.95, 0.98, 0.99, 0.995,0.999]),
        'kl_target': tune.choice([0.001,0.01,0.1]),
  }


def mutate_ddpg_hyperparams():
  
  return {
  "buffer_size": [int(1e4), int(1e5), int(1e6)],
  "lr": lambda: loguniform(1e-5, 1),
  "train_batch_size": [32, 64, 128, 256, 512]
  }

def mutate_ppo_hyperparams():

  return {
    "entropy_coeff": lambda: loguniform(0.00000001, 0.1),
      "lr": lambda: loguniform(5e-5, 0.0001),
      "sgd_minibatch_size": [ 32, 64, 128, 256, 512],
      "lambda": [0.9, 0.95, 0.98, 0.99, 0.995,0.999],
      'clip_param': [0.1,0.2,0.3,0.4],
      "vf_loss_coeff": lambda: np.random.uniform(0,1),
      'kl_target': [0.001,0.01,0.1]

  }
  
def mutate_a2c_hyperparams():
  return{
      "lambda": [0.1,0.3,0.5,0.7,0.9,1.0],
    "entropy_coeff":  lambda: loguniform(0.00000001, 0.1),
    "lr":  lambda: loguniform(1e-5, 1) 
}

def a2c_params_bounds():
  return {
  "buffer_size": [int(1e4), int(1e6)],
  "lr": [1e-5, 1],
  "train_batch_size": [32, 512]
  }

def ppo_param_bounds():
  return {
    'lr':[5e-5, 0.0001],
    'sgd_minibatch_size': [64,256],
    'entropy_coeff': [0.00000001, 0.1],
      'clip_param': [0.1,0.4],
      "vf_loss_coeff": [0,1],
      'lambda': [0.9,0.999],
      'kl_target': [0.001,0.1], 
  }

def ddpg_param_bounds():
  return {
    "buffer_size": [int(1e4), int(1e6)],
    "lr": [1e-5, 1],
    "train_batch_size": [32, 512]
  }


# Running Tune

In [ ]:
MODEL_TRAINER = {'a2c':A2CTrainer,'ppo':PPOTrainer,'ddpg':DDPGTrainer}

In [ ]:
num_samples = 1 #Population size
training_iterations = 1 #Number of times the metric has been reported

In [ ]:
if model_name == "ddpg":
    sample_hyperparameters = sample_ddpg_params()
    mutate_hyperparameters = mutate_ddpg_hyperparams()
    params_bounds = ddpg_param_bounds()
elif model_name == "ppo":
  sample_hyperparameters = sample_ppo_params()
  mutate_hyperparameters = mutate_ppo_hyperparams()
  params_bounds = ppo_param_bounds()
elif model_name == "a2c":
  sample_hyperparameters = sample_a2c_params()
  mutate_hyperparameters = mutate_a2c_hyperparams()
  params_bounds = a2c_params_bounds()

In [ ]:

pbt_scheduler = PopulationBasedTraining(
    time_attr = "training_iteration",
    perturbation_interval = training_iterations/5,
    burn_in_period = 0.0,
    quantile_fraction = 0.25,
    hyperparam_mutations = mutate_hyperparameters
)

pb2_scheduler = PB2(
                    time_attr='training_iteration',            
                    perturbation_interval=training_iterations/5,
                    quantile_fraction= 0.25,
                    hyperparam_bounds={
                       **params_bounds
                    })

def run_PBT(log_dir):

  analysis = tune.run(
      MODEL_TRAINER[model_name],
      scheduler=pbt_scheduler, #To prune bad trials
      metric='episode_reward_mean',
      mode='max',
      config = {**sample_hyperparameters,
                'env':'StockTrading_train_env','num_workers':1,
                'num_gpus':1,'framework':'torch','log_level':'DEBUG'},
      num_samples = num_samples, #Number of hyperparameters to test out
      stop = {'training_iteration':training_iterations},#Time attribute to validate the results
      verbose=1,local_dir="./"+log_dir,#Saving tensorboard plots
      resources_per_trial={'gpu':1,'cpu':1},
      max_failures = 1,#Extra Trying for the failed trials
      raise_on_failed_trial=False,#Don't return error even if you have errored trials
      # keep_checkpoints_num = 2, 
      checkpoint_score_attr ='episode_reward_mean',#Only store keep_checkpoints_num trials based on this score
      checkpoint_freq=training_iterations,#Checpointing all the trials,
  )
  # print("Best hyperparameter: ", analysis.best_config)
  return analysis

def run_PB2(log_dir):

  analysis = tune.run(
      MODEL_TRAINER[model_name],
      scheduler=pb2_scheduler, #To prune bad trials
      metric='episode_reward_mean',
      mode='max',
      config = {**sample_hyperparameters,
                'env':'StockTrading_train_env','num_workers':1,
                'num_gpus':1,'framework':'torch','log_level':'DEBUG'},
      num_samples = num_samples, #Number of hyperparameters to test out
      stop = {'training_iteration':training_iterations},#Time attribute to validate the results
      verbose=1,local_dir="./"+log_dir,#Saving tensorboard plots
      resources_per_trial={'gpu':1,'cpu':1},
      max_failures = 1,#Extra Trying for the failed trials
      raise_on_failed_trial=False,#Don't return error even if you have errored trials
      # keep_checkpoints_num = 2, 
      checkpoint_score_attr ='episode_reward_mean',#Only store keep_checkpoints_num trials based on this score
      checkpoint_freq=training_iterations,#Checpointing all the trials
  # print("Best hyperparameter: ", analysis.best_config)
  return analysis

In [ ]:
# analysis = run_PB2("PB2_Tuned")
analysis = run_PB2("PB2_Tuned")

In [ ]:
# !zip -r /content/PBT_Tuned.zip /content/PBT_Tuned
# from google.colab import files
# files.download("/content/PBT_Tuned.zip")

## Best config, hyperparameters and checkpoints


In [ ]:
best_config = analysis.get_best_config(metric='episode_reward_mean',mode='max')
best_config

{'clip_param': 0.1,
 'entropy_coeff': 2.7984540504890533e-08,
 'env': 'StockTrading_train_env',
 'framework': 'torch',
 'kl_target': 0.1,
 'lambda': 0.95,
 'log_level': 'DEBUG',
 'lr': 8.016314885885411e-05,
 'num_gpus': 1,
 'num_workers': 1,
 'sgd_minibatch_size': 64,
 'vf_loss_coeff': 0.2489739417927158}

In [ ]:
best_logdir = analysis.get_best_logdir(metric='episode_reward_mean',mode='max')
best_logdir

'/content/PB2_Tuned/PPOTrainer_2022-07-10_21-59-13/PPOTrainer_StockTrading_train_env_89281_00000_0_clip_param=0.1000,entropy_coeff=0.0000,kl_target=0.1000,lambda=0.9500,lr=0.0001,sg_2022-07-10_21-59-15'

In [ ]:
best_checkpoint = analysis.best_checkpoint

### TESTING

In [ ]:
%%capture
!unzip "/content/PB2_Tuned.zip"
!unzip "/content/PBT_Tuned.zip"

In [ ]:
test_config = get_test_config(TEST_START_DATE, TEST_END_DATE, ticker_list, data_source, time_interval, 
         technical_indicator_list, env, model_name, if_vix = True)

test_env_name = 'StockTrading_test_env'
register_env(env_name, lambda config: env(test_config))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
# import json
# chkpt_dir = "/content/content/PB2_Tuned/PPOTrainer_2022-06-30_18-03-49/PPOTrainer_StockTrading_train_env_fe24f_00006_6_clip_param=0.2000,entropy_coeff=0.0323,kl_target=0.1000,lambda=0.9950,lr=0.0001,sg_2022-06-30_18-09-42"

# with open(chkpt_dir+'/params.json') as f:
#   model_config = json.load(f)

# model_config['env'] = test_env_name
# print(model_config)
# trainer = ppo.PPOTrainer(config=model_config)
# trainer.restore(chkpt_dir)

## Testing

In [ ]:
def DRL_prediction(
        model_name,
        test_env_config,
        env,
        model_config,
        agent_path,
        env_name_test='StockTrading_test_env'
    ):

        env_instance = env(test_env_config)
        
        register_env(env_name_test, lambda config: env(test_env_config))
        model_config['env'] = env_name_test
        # ray.init() # Other Ray APIs will not work until `ray.init()` is called.
        if model_name == "ppo":
            trainer = MODELS[model_name].PPOTrainer(config=model_config)
        elif model_name == "a2c":
            trainer = MODELS[model_name].A2CTrainer(config=model_config)
        elif model_name == "ddpg":
            trainer = MODELS[model_name].DDPGTrainer(config=model_config)
        elif model_name == "td3":
            trainer = MODELS[model_name].TD3Trainer(config=model_config)
        elif model_name == "sac":
            trainer = MODELS[model_name].SACTrainer(config=model_config)

        try:
            trainer.restore(agent_path)
            print("Restoring from checkpoint path", agent_path)
        except BaseException:
            raise ValueError("Fail to load agent!")

        # test on the testing env
        state = env_instance.reset()
        episode_returns = list()  # the cumulative_return / initial_account
        episode_total_assets = list()
        episode_total_assets.append(env_instance.initial_total_asset)
        done = False
        while not done:
            action = trainer.compute_single_action(state)
            state, reward, done, _ = env_instance.step(action)

            total_asset = (
                env_instance.amount
                + (env_instance.price_ary[env_instance.day] * env_instance.stocks).sum()
            )
            episode_total_assets.append(total_asset)
            episode_return = total_asset / env_instance.initial_total_asset
            episode_returns.append(episode_return)
        ray.shutdown()
        print("episode return: " + str(episode_return))
        print("Test Finished!")
        return episode_total_assets


In [ ]:
episode_total_assets = DRL_prediction(
        model_name,
        test_config,
        env,
        best_config,
        '/content/PB2_Tuned/PPOTrainer_2022-07-10_21-59-13/PPOTrainer_StockTrading_train_env_89281_00000_0_clip_param=0.1000,entropy_coeff=0.0000,kl_target=0.1000,lambda=0.9500,lr=0.0001,sg_2022-07-10_21-59-15/checkpoint_000001/checkpoint-1',
        env_name_test='StockTrading_test_env')

2022-07-10 22:03:58,436	INFO ppo.py:415 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(RolloutWorker pid=8554) 2022-07-10 22:04:12,767	WARNING env.py:136 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=8554) 2022-07-10 22:04:12,781	DEBUG rollout_worker.py:1770 -- Creating policy for default_policy
(RolloutWorker pid=8554) 2022-07-10 22:04:12,789	DEBUG catalog.py:807 -- Created preprocessor <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7fdd576afe50>: Box([-3000. -3000. -3000. -3000. -3000. -3000. -3000. -3000. -3000. -3000.
(RolloutWorker pid=8554)  -3000. -3000. -3000. -3000. -3000. -3000. -3000. -3000. -3000. -3000.
(RolloutWorker pid=8554)  -3

Restoring from checkpoint path /content/PB2_Tuned/PPOTrainer_2022-07-10_21-59-13/PPOTrainer_StockTrading_train_env_89281_00000_0_clip_param=0.1000,entropy_coeff=0.0000,kl_target=0.1000,lambda=0.9500,lr=0.0001,sg_2022-07-10_21-59-15/checkpoint_000001/checkpoint-1
episode return: 1.139153132802156
Test Finished!


In [ ]:
calculate_sharpe(episode_total_assets)

0.7742295439858137